In [26]:
from PIL import Image, ImageEnhance
import numpy as np
import os

In [27]:
def remove_night_flare(img_path, output_folder):
    # Read image
    img = Image.open(img_path)
    hh, ww = img.size

    # Convert image to NumPy array
    img_np = np.array(img)

    # Threshold
    lower = (150, 150, 150)
    upper = (240, 240, 240)
    night_flare_mask = np.logical_and.reduce((
        img_np[:, :, 0] >= lower[0],
        img_np[:, :, 1] >= lower[1],
        img_np[:, :, 2] >= lower[2],
        img_np[:, :, 0] <= upper[0],
        img_np[:, :, 1] <= upper[1],
        img_np[:, :, 2] <= upper[2]
    ))

    # Convert to uint8
    night_flare_mask = (night_flare_mask * 255).astype(np.uint8)

    # Apply brightness and contrast adjustment
    brightness_factor = 0.5  # Adjust as needed
    contrast_factor = 1.5    # Adjust as needed

    enhancer = ImageEnhance.Brightness(img)
    img_brightness = enhancer.enhance(brightness_factor)

    enhancer = ImageEnhance.Contrast(img_brightness)
    img_no_flare = enhancer.enhance(contrast_factor)

    # Save results
    img_name = os.path.basename(img_path)
    #flare_mask_path = os.path.join(output_folder, f"{img_name}_flare_mask.jpg")
    img_no_flare_path = os.path.join(output_folder, f"{img_name}")

    #Image.fromarray(night_flare_mask).save(flare_mask_path)
    img_no_flare.save(img_no_flare_path)


In [28]:
# Directory containing input images
input_folder = r'C:\Users\vickj\Documents\Computer Vision_Sem2_Mtech\NigntFlare_Removal_MIPI\Valid\val_input_2k_bicubic'

# Output folder for processed images
output_folder = r'C:\Users\vickj\Documents\Computer Vision_Sem2_Mtech\NigntFlare_Removal_MIPI\Output2'


In [29]:
# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each image in the input folder
for img_file in os.listdir(input_folder):
    if img_file.endswith(".png") or img_file.endswith(".jpg"):
        img_path = os.path.join(input_folder, img_file)
        remove_night_flare(img_path, output_folder)

In [9]:
from PIL import Image, ImageEnhance
import cv2
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider

def threshold_image_color_space(img_np, lower=(150, 150, 150), upper=(240, 240, 240), color_space=cv2.COLOR_RGB2LAB):
    img_np_color = cv2.cvtColor(img_np, color_space)
    return cv2.inRange(img_np_color, np.array(lower), np.array(upper))

def adaptive_threshold_image(img_np, block_size=11, c=2, color_space=cv2.COLOR_RGB2LAB):
    img_np_color = cv2.cvtColor(img_np, color_space)
    gray_img = cv2.cvtColor(img_np_color, cv2.COLOR_BGR2GRAY)
    return cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)

def adjust_brightness_contrast(img, brightness_factor=0.5, contrast_factor=1.5):
    enhancer = ImageEnhance.Brightness(img)
    img_result = enhancer.enhance(brightness_factor * contrast_factor)
    return img_result

def enhance_color_channels(img_np, factor=1.5):
    img_np[:, :, 0] = np.clip(img_np[:, :, 0] * factor, 0, 255)
    img_np[:, :, 1] = np.clip(img_np[:, :, 1] * factor, 0, 255)
    img_np[:, :, 2] = np.clip(img_np[:, :, 2] * factor, 0, 255)
    return img_np

def combine_masks(mask1, mask2):
    return cv2.bitwise_or(mask1, mask2)

def refine_mask(mask):
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=1)
    return mask

def threshold_tuner(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots()
    plt.subplots_adjust(bottom=0.25)

    ax_lower = plt.axes([0.1, 0.01, 0.65, 0.03])
    ax_upper = plt.axes([0.1, 0.06, 0.65, 0.03])

    s_lower = Slider(ax_lower, 'Lower', 0, 255, valinit=150)
    s_upper = Slider(ax_upper, 'Upper', 0, 255, valinit=240)

    def update(val):
        lower_threshold = (s_lower.val, s_lower.val, s_lower.val)
        upper_threshold = (s_upper.val, s_upper.val, s_upper.val)

        night_flare_mask1 = adaptive_threshold_image(img, block_size=11, c=2)
        night_flare_mask2 = threshold_image_color_space(img, lower_threshold, upper_threshold)
        refined_mask = combine_masks(night_flare_mask1, night_flare_mask2)
        refined_mask = refine_mask(refined_mask)

        ax.imshow(np.where(refined_mask[:, :, None], img, 255), cmap='gray')

    s_lower.on_changed(update)
    s_upper.on_changed(update)

    plt.show()

def remove_night_flare(img_path, output_folder, block_size=11, c=2, brightness_factor=0.8, contrast_factor=1.5):
    img = Image.open(img_path)
    img_np = np.array(img)

    img_np = enhance_color_channels(img_np)

    night_flare_mask1 = adaptive_threshold_image(img_np, block_size=block_size, c=c)
    night_flare_mask2 = threshold_image_color_space(img_np)

    night_flare_mask = combine_masks(night_flare_mask1, night_flare_mask2)
    refined_mask = refine_mask(night_flare_mask)

    img_result = adjust_brightness_contrast(img, brightness_factor, contrast_factor)

    img_name = os.path.basename(img_path)
    img_no_flare_path = os.path.join(output_folder, img_name)

    img_result.save(img_no_flare_path)

# Directory containing input images
input_folder = r'C:\Users\vickj\Documents\Computer Vision_Sem2_Mtech\NigntFlare_Removal_MIPI\Valid\val_input_2k_bicubic'

# Output folder for processed images
output_folder = r'C:\Users\vickj\Documents\Computer Vision_Sem2_Mtech\NigntFlare_Removal_MIPI\Output2'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each image in the input folder
for img_path in glob.glob(os.path.join(input_folder, "*.png")) + glob.glob(os.path.join(input_folder, "*.jpg")):
    remove_night_flare(img_path, output_folder)
